In [1]:
import pandas as pd

In [2]:
accounts = pd.read_csv("accounts.csv")
accounts.head()

,name,id,category,url,include?,Twitter page,stats,stats_type
0,A.J. Klein,120578461444970,Athlete,http://facebook.com/120578461444970,N,NaN,1511.0,Y
1,Aaron Brackett,169056606481397,Public Figure,http://facebook.com/169056606481397,Y,https://twitter.com/Aaron_Brackett,473.0,N
2,Aaron Gilchrist,99145336302,Journalist,http://facebook.com/99145336302,Y,https://twitter.com/nbcaaron,1531.0,Twitter
3,Aarón Sánchez,106075582777270,Chef,http://facebook.com/106075582777270,Y,https://twitter.com/Chef_Aaron,NaN,NaN
4,Abby Huntsman,382496035168964,Public Figure,http://facebook.com/382496035168964,N,NaN,NaN,NaN


In [3]:
fb = accounts.loc[accounts['include?'] =='Y', ["name", "id"]]
fb.head()

,name,id
1,Aaron Brackett,169056606481397
2,Aaron Gilchrist,99145336302
3,Aarón Sánchez,106075582777270
5,ABC13-Christine Dobbyn,149105561803589
6,ABC13-Dave Ward,469939783076025


In [4]:
fb.count()

name    1511
id      1511
dtype: int64

In [5]:
fb.to_csv("facebook.csv", index = False)

In [6]:
twitter = accounts.loc[(accounts['include?'] == 'Y') & accounts['Twitter page'].notnull(), "Twitter page"]
twitter.head()

1     https://twitter.com/Aaron_Brackett
2           https://twitter.com/nbcaaron
3         https://twitter.com/Chef_Aaron
5    https://twitter.com/ChristineDobbyn
6      https://twitter.com/DaveWardABC13
Name: Twitter page, dtype: object

In [7]:
twitter.count()

1509

In [8]:
import tweepy
import yaml
file = open("api-keys.yml", "r")
api_keys = yaml.load(file)
file.close()
auth = tweepy.OAuthHandler(api_keys["twitter"]["consumer-key"],
                           api_keys["twitter"]["consumer-secret"])
auth.set_access_token(api_keys["twitter"]["access-token"], 
                      api_keys["twitter"]["access-secret"])
api = tweepy.API(auth)

In [9]:
names = [page.split('/')[-1] for page in twitter]
len(names)

1509

In [10]:
import time
ids = []
for n, name in enumerate(names):
    if n > 899 and n % 900 == 0:
        time.sleep(15*60)
    try: 
        user_id = api.get_user(screen_name = name).id
        ids.append((name, user_id))
    except:
        continue

In [11]:
len(ids)

1500

In [14]:
names_ids = [values for values in zip(*ids)]
twitter = pd.DataFrame({'screen_name': names_ids[0], 'id': names_ids[1]})[["screen_name", "id"]]
twitter.head()

,screen_name,id
0,Aaron_Brackett,34777103
1,nbcaaron,235216696
2,Chef_Aaron,58816328
3,ChristineDobbyn,367948574
4,DaveWardABC13,1392493334


In [15]:
twitter.to_csv('twitter.csv', index = False)